In [1]:
import sys
sys.path.append('..')
from itertools import permutations
from modified_sage_funcs import *
from imp_funcs import *

n = 5
M = matroids.CompleteGraphic(7)
r = M.rank()
S = (frozenset({0, 1, 2, 3, 4}), frozenset({2, 3, 4, 5, 6, 7}))
k = len(S)
P = PolynomialRing(QQ, 'x', k)

In [2]:


def partitions(basis, weak_comp):
    """
    partitions a basis into weak components
    """
    parts = set()
    for perm in permutations(basis):
        partition = []
        start = 0
        for size in weak_comp:
            partition.append(frozenset(perm[start:start + size]))
            start += size
        parts.add(tuple(partition))
    return tuple(parts)

def base_partitions(bases, indexed_compositions):
    """
    partitions each basis into weak components
    """
    bases_partitions = {}
    for basis in bases:
        base_partitions = {}
        for i in indexed_compositions:
            base_partitions[i] = partitions(tuple(basis), indexed_compositions[i])
        bases_partitions[basis] = base_partitions
    return bases_partitions


In [3]:
def composition_weight(iwc, set_system, bp):
    """
    computes the weight of each composition
    """
    weights = {}
    for i in iwc:
        weight = 0
        for basis in bp:
            for basis_partition in bp[basis][i]:
                flag = True
                for x, y, z in zip(iwc[i], basis_partition, set_system):
                    if len(y.intersection(z)) == x:
                        continue
                    else:
                        flag = False
                        break
                if flag:
                    weight += 1
                    break
        weights[i] = weight
    return weights

In [4]:
def weighted_poly(M, set_system):
    d, n = M.full_rank(), len(M.groundset())
    k = len(set_system)
    P = PolynomialRing(QQ, 'x', k)
    bases = sorted([frozenset(sorted(X)) for X in bases_iterator(M)])
    indexed_compositions = {i : x for i, x in enumerate(weak_compositions(d, k))}
    bp = base_partitions(bases, indexed_compositions)
    weights = composition_weight(indexed_compositions, set_system, bp)
    def monomial(wc):
        return prod([P.gen(i)**wc[i] for i in range(len(wc))])
    def weighted_sum(weights):
        return sum([weights[i]*monomial(indexed_compositions[i]) for i in weights])
    return weighted_sum(weights)

In [5]:
def decompose(S):
    return tuple(frozenset([x]) for sets in S for x in sets)

indexed_compositions = {i : x for i, x in enumerate(weak_compositions(r, k))}
indexed_compositions

{0: [1, 5], 1: [2, 4], 2: [3, 3], 3: [4, 2], 4: [5, 1], 5: [6, 0], 6: [0, 6]}

In [ ]:
Sp = weighted_poly(M, S)
dSp = weighted_poly(M, decompose(S))

print(Sp)
print(dSp)

In [ ]:
def specialize(f, S1, S2):
    P1 = PolynomialRing(QQ, 'x', len(S1))
    P2 = PolynomialRing(QQ, 'x', len(S2))
    k = 0
    for i, x in enumerate(S1):
        subs_dict = {P2.gen(k + j): P1.gen(i) for j in range(len(x))}
        f = f.subs(subs_dict)
        k += len(x)
    return f

print(specialize(dSp, S, decompose(S)))

3*x0^4 + 18*x0^3*x1 + 34*x0^2*x1^2 + 22*x0*x1^3 + 4*x1^4


In [ ]:
def coefficients_dict(f):
    return {f.exponents()[i]: f.coefficients()[i] for i in range(len(f.coefficients()))}

coefficients_dict(specialize(dSp, S, decompose(S)))

{(4, 0, 0, 0, 0, 0, 0, 0, 0, 0): 3,
 (3, 1, 0, 0, 0, 0, 0, 0, 0, 0): 18,
 (2, 2, 0, 0, 0, 0, 0, 0, 0, 0): 34,
 (1, 3, 0, 0, 0, 0, 0, 0, 0, 0): 22,
 (0, 4, 0, 0, 0, 0, 0, 0, 0, 0): 4}

In [ ]:
print(Sp)
coefficients_dict(Sp)

3*x0^4 + 13*x0^3*x1 + 20*x0^2*x1^2 + 16*x0*x1^3 + 4*x1^4


{(4, 0): 3, (3, 1): 13, (2, 2): 20, (1, 3): 16, (0, 4): 4}

In [ ]:
def normalize(f):
    return sum(f.monomials()[i] * f.coefficients()[i]/prod([factorial(x) for x in f.exponents()[i]]) for i in range(len(f.coefficients())))

nSp = normalize(Sp)
nSp

1/8*x0^4 + 13/6*x0^3*x1 + 5*x0^2*x1^2 + 8/3*x0*x1^3 + 1/6*x1^4

In [ ]:
nSp.is_lorentzian()

True